In [1]:
import boto3
import json
import time
from dotenv import load_dotenv
from datetime import datetime
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [3]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [4]:

# Function to get the shard iterator
def get_shard_iterator(stream_name, shard_id):
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType='LATEST'  # or 'LATEST' for new records
    )
    return response['ShardIterator']

# Function to read records from the Kinesis stream
def read_kinesis_records(stream_name, shard_iterator):
    while True:
        response = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
        records = response['Records']
        for record in records:
            # No need to base64 decode, just parse the data directly
            order_data = record['Data']
            order = json.loads(order_data)
            print("Received order:", order)

        # Update the shard iterator for the next batch of records
        shard_iterator = response['NextShardIterator']

        # Sleep to avoid hitting API rate limits
        time.sleep(1)


In [5]:
# Get the shard iterator
stream_name = 'OrderStreamForDispatching'  # Replace with your stream name
shard_id = 'shardId-000000000000'  # Get the shard ID from the stream's details

shard_iterator = get_shard_iterator(stream_name, shard_id)

# Start reading records
read_kinesis_records(stream_name, shard_iterator)

Received order: {'event_id': 'de292515-a4c7-4ed0-91c4-87d890e7ccc0', 'event_type': 'ORDER_CREATED', 'event_timestamp': '2024-10-13 23:43:22', 'order_id': '4c4082fc-c3aa-41ce-a3a1-41105a1b3def', 'order_details': {'customer_id': 'cus-93ba3752-3840-443d-ba2b-0b6034458994', 'order_timestamp': '2024-10-13 23:43:22', 'order_date': '2024-10-13', 'items': [{'product_id': '973f7b7e-0e91-47a9-ac88-801f17ca9f6a', 'product_name': 'illum', 'price': 129.7480125782148, 'quantity': 2, 'packages': [{'package_id': '6422383d-5e97-4511-87cb-b842038f2117', 'subpackage_id': 73, 'quantity': 2, 'weight': 8.395773678895077, 'volume': 1.7482534213944958}, {'package_id': 'c781f4a3-3edd-4d39-9890-12ea7073e226', 'subpackage_id': 59, 'quantity': 1, 'weight': 4.678758546199668, 'volume': 3.331867937025425}, {'package_id': '0e31bf25-dd2d-4d07-a2c3-1644bc16b8a4', 'subpackage_id': 89, 'quantity': 2, 'weight': 1.1290095042006996, 'volume': 0.3375527767259017}]}, {'product_id': '9a3d63ef-38f4-4657-9d38-ff78f849c07e', 'pr

KeyboardInterrupt: 